In [1]:
! pip -q install datasets bitsandbytes accelerate xformers einops torch torchvision==0.19.1 transformers==4.45.2

In [2]:
import torch
import transformers
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
good_prompt = """
# Recipe Assistant

## Context
You are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.

## Retrieved Recipes
The following recipes have been retrieved as most relevant to the user's query:

{retrieved_recipes}

## Instructions
1. **Answer directly from the retrieved recipes when possible.** Use the information from the provided recipes to answer questions about ingredients, cooking methods, nutritional information, and preparation steps.

2. **For ingredient questions:**
   - Provide accurate amounts and measurements from the recipes
   - Suggest possible substitutions based on common culinary knowledge
   - Explain the purpose of key ingredients in the dish

3. **For cooking technique questions:**
   - Explain preparation methods mentioned in the recipes
   - Clarify cooking times and temperatures
   - Describe expected results and how to tell when food is properly cooked

4. **For modification requests:**
   - Suggest appropriate adjustments for dietary restrictions (vegan, gluten-free, etc.)
   - Explain how to scale recipes up or down
   - Offer ideas for flavor variations while maintaining the core identity of the dish

5. **For general questions:**
   - Provide brief culinary background/history when relevant
   - Explain unfamiliar cooking terms
   - Suggest pairings, serving suggestions, and storage recommendations

## Response Format
- Start with a direct answer to the user's question
- Keep your responses concise but comprehensive
- For multi-step instructions or complex concepts, organize information in a clear, logical structure
- If the retrieved recipes don't contain sufficient information to answer the query, acknowledge the limitations and provide general culinary knowledge that might help
- When suggesting modifications not explicitly in the retrieved recipes, clearly indicate these are your recommendations based on culinary principles

## Limitations
- Don't make claims about specific nutritional values unless they're mentioned in the retrieved recipes
- If asked about topics completely unrelated to cooking or the recipes provided, politely redirect the conversation back to recipe-related topics
- Don't invent or fabricate details about recipes that aren't in the retrieved data

## User Query
{user_query}
"""


bad_prompt = """
You are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.

The following recipes have been retrieved as most relevant to the user's query:
{retrieved_recipes}

## User Query
{user_query}
"""


irrelevant_context = """
Richard Gary Brautigan (January 30, 1935 – c. September 16, 1984)
was an American novelist, poet, and short story writer. A prolific writer,
he wrote throughout his life and published ten novels, two collections of
short stories, and four books of poetry. Brautigan's work has been published
both in the United States and internationally throughout Europe, Japan,
and China. He is best known for his novels Trout Fishing in America (1967),
In Watermelon Sugar (1968), and The Abortion: An Historical Romance 1966 (1971).
"""


prompt_with_scores = "\n\n# Recipe Assistant\n\n## Context\nYou are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.\n\n## Retrieved Recipes\nThe following recipes have been retrieved as most relevant to the user's query:\n\nDocument 0, Score: 0.5509\nName: semi copycat sporky pizza bertucci\nDescription: this was my favorite pizza when i lived in boston.  ive tried to get our local pizza places to make and sell it, but to no avail. they just make the same old - same old pizza!\r\ni have not included oven temp or time to cook because it all depends on the pizza you buy.\nIngredients: frozen cheese pizza, tomatoes, ricotta cheese, mozzarella cheese, olive oil, italian sausages\nSteps: set the oven at the right temp from directions on pizza box, sprinkle part of olive oil over pizza top, slice tomato and spread on top of pizza, use the riccota in chunks and spread over pizza, spread chunks of sausage over top of pizza, sprinkle rest of olive oil over all, top with as much mozzorella as you wish, put into oven for as long as required by pizza directions on box, however , 5 minutes before the minimum time has elapsed , check to see if pizza is ready\n\nDocument 1, Score: 0.5456\nName: skinny pizza\nDescription: when you want pizza but not all the calories and fat!\nIngredients: rice cakes, fat-free american cheese slices, pizza sauce, pizza seasoning, basil, vegetables\nSteps: put pizza sauce on rice cakes, sprinkle pizza seasoning , basil and any optional vegetables on top of pizza, cover each pizza with a piece of fat free cheese, heat in microwave for 30-60 sec, depending on your preference\n\nDocument 2, Score: 0.5448\nName: pizza con le patate potato pizza\nDescription: potato pizza\nIngredients: pizza dough, potatoes, rosemary, extra virgin olive oil, salt and pepper\nSteps: preheat oven to 500 f, evenly spread the potato slices over the pizza, top with the rosemary leaves, spread the olive oil on the pizza, add salt and pepper, bake the pizza for approximately 10 - 15 minutes, lift one side to check for readiness, pizza is ready when the bottom surface is light brown\n\nDocument 3, Score: 0.5420\nName: make pizza\nDescription: pizza was originally started in naples, italy but has now become a popular type of food, it can go with almost anything and creating your own could taste better than the ones you buy in the shop. this article will give you 2 different ways of creating your pizza, one method is faster than the other.!\nIngredients: tomato paste, ham, cheese, olive, sausage\nSteps: preheat your oven to about 180 degrees, buy ready-made pizza bases , such as mccains ready-made pizza bases if you are short on time, if you have plenty of time , then you can just get dough, to make the method faster , dont get a huge pizza if youre only serving yourself or two, you dont want to take forever making your pizza, spread tomato paste on your pizza base, it is a good idea to buy one that is specially made for pizzas, you may want to sprinkle some italian seasoning with the sauce as well, make sure you dont put tomato base on the outside part of the pizza , or you wont have a crust for your pizza, dont make the sauce layer too thick or too thin, put a layer of ham onto your pizza, use nice , wholesome and delicious ham for your pizza to help it taste better, if a brand claims that it is the best ham , buy it , taste it , and see for yourself, in order to find the best ham , take a whole bunch of ham from the store, it doesnt have to be the perfect ham , just the ind of ham that you like, some people prefer smoked , while others prefer cooked ham , or honey ham, it all depends on you , and a couple of other people to help take votes, make sure that it is within its use-by date, ham is optional because some people dont like ham in their pizza, add a sprinkling of cheese, you can put as little or as much as you like , depending on your personal taste, mozzarella pizza cheese usually tastes really nice on pizzas, other people prefer other types of cheese, just like the ham , it is up to you, make sure that there are no spots without the cheese for your best taste, some people dont like the cheese , while some dont want too much cheese, common pizza takes the same old mozzarella , but if you are feeling creative , you can use any cheese you want-with any amount ! try being creative if you are artistic , and if you area serious chef then cook the traditional way, the benefits of creating your own is that you can find a better tasting way of pizza , and it is more fun !, enjoy it by adding sliced olives to your pizza but make sure that the olives are pitless, you dont have to have olives , this is optional, you can take almost any topping you want , like extra cheese , sausage , meatballs , pepperoni , chili peppers , supreme , veggies , and others, remember that it is your pizza and that you can do whatever you want with it, this time , youre the chef !, place your pizza on an oven tray sprayed with olive-oil , so that the pizza doesnt stick, you can use anything that prevents the pizza from sticking to the pan, this is important , and if you dont do that , you can end up without the pizza, put your pizza in the oven , and turn it down to about 160 degrees, too hot pizzas can burn you , and then youll have to wait to eat your pizza, you dont want to have to wait for the pizza to cool off, plus , you can get distracted and leave it in too long , then realize that you forgot the pizza was in the oven , and then you have charcoal in the oven, nobody wants their work to turn into burn ashes , dont they ? this happens with some people, take it out after about 15 - 25 minutes , depending on your oven, use your own judgment to tell when the pizza is ready, the cheese should be a golden brown , but it should not be burnt, grill method spread the sauce all over the tortilla but you might want to leave a little crust around the edge, leaving some around the edge will give you that delicious , crispy pizza taste, also , it will give you that nice crust to eat later after the main part of the pizza is eaten, put on your toppings and arrange, remember , you can take any topping you want , anywhere you want, just dont melt a pepperoni on a meatball !, grate the cheese over the pizza, you can use basically any cheese such as cheddar or edam, mozzarella cheese is the best for pizzas and it is the most common , but there are many cheeses to choose from, simply put it under the grill, be careful not to get burned and your pizza could sizzle and pop because of the common bubbles in the pizza crust, the bubbles form often in pizza , but the best pizza doesnt have bubbles, take it out after three minutes , it should be done ! now you can enjoy your tasty work, wood-fired oven method get your pizza base , it can be any sort , although it cant be more than 2 cms thick, place your tomato base and toppings on , do not be scared to put sausage , or any other raw meat on, it will cook, make sure your wood-fired oven is very hot for the best pizza ever and to help the pizza cook faster, put your pizza into the oven , preferably on a tray so the base does not burn, remember that you dont want your work to burn , and nobody wants a burnt pizza, turn your pizza every 30 seconds for two minutes , and at the 1:30 mark take it off the tray, now your pizza should be done, take your pizza out and enjoy !\n\nDocument 4, Score: 0.5161\nName: margherita pizza white pizza\nDescription: tomato, basil, garlic, olive oil, mozzarella and ricotta cheese.\nIngredients: pizza crust, olive oil, pizza cheese, tomatoes, fresh basil, ricotta cheese, garlic powder\nSteps: brush pizza crust with olive oil, spread 2 cups of cheese evenly over pizza, place sliced tomato evenly over pizza, spread shredded basil evenly over pizza, place tablespoon size globs of ricotta evenly over pizza, sprinkle with garlic powder, spread remainig mozzarella over pizza, bake at 400 15 minute or until cheese is bubbly and brown on the edges\n\n\n\n## Instructions\n1. **Answer directly from the retrieved recipes when possible.** Use the information from the provided recipes to answer questions about ingredients, cooking methods, nutritional information, and preparation steps.\n\n2. **For ingredient questions:**\n   - Provide accurate amounts and measurements from the recipes\n   - Suggest possible substitutions based on common culinary knowledge\n   - Explain the purpose of key ingredients in the dish\n\n3. **For cooking technique questions:**\n   - Explain preparation methods mentioned in the recipes\n   - Clarify cooking times and temperatures\n   - Describe expected results and how to tell when food is properly cooked\n\n4. **For modification requests:**\n   - Suggest appropriate adjustments for dietary restrictions (vegan, gluten-free, etc.)\n   - Explain how to scale recipes up or down\n   - Offer ideas for flavor variations while maintaining the core identity of the dish\n\n5. **For general questions:**\n   - Provide brief culinary background/history when relevant\n   - Explain unfamiliar cooking terms\n   - Suggest pairings, serving suggestions, and storage recommendations\n\n## Response Format\n- Start with a direct answer to the user's question\n- Keep your responses concise but comprehensive\n- For multi-step instructions or complex concepts, organize information in a clear, logical structure\n- If the retrieved recipes don't contain sufficient information to answer the query, acknowledge the limitations and provide general culinary knowledge that might help\n- When suggesting modifications not explicitly in the retrieved recipes, clearly indicate these are your recommendations based on culinary principles\n\n## Limitations\n- Don't make claims about specific nutritional values unless they're mentioned in the retrieved recipes\n- If asked about topics completely unrelated to cooking or the recipes provided, politely redirect the conversation back to recipe-related topics\n- Don't invent or fabricate details about recipes that aren't in the retrieved data\n\n## User Query\nHow do you make pizza\n"


prompt_without_scores = "\n\n# Recipe Assistant\n\n## Context\nYou are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.\n\n## Retrieved Recipes\nThe following recipes have been retrieved as most relevant to the user's query:\n\nDocument 0\nName: semi copycat sporky pizza bertucci\nDescription: this was my favorite pizza when i lived in boston.  ive tried to get our local pizza places to make and sell it, but to no avail. they just make the same old - same old pizza!\r\ni have not included oven temp or time to cook because it all depends on the pizza you buy.\nIngredients: frozen cheese pizza, tomatoes, ricotta cheese, mozzarella cheese, olive oil, italian sausages\nSteps: set the oven at the right temp from directions on pizza box, sprinkle part of olive oil over pizza top, slice tomato and spread on top of pizza, use the riccota in chunks and spread over pizza, spread chunks of sausage over top of pizza, sprinkle rest of olive oil over all, top with as much mozzorella as you wish, put into oven for as long as required by pizza directions on box, however , 5 minutes before the minimum time has elapsed , check to see if pizza is ready\n\nDocument 1\nName: skinny pizza\nDescription: when you want pizza but not all the calories and fat!\nIngredients: rice cakes, fat-free american cheese slices, pizza sauce, pizza seasoning, basil, vegetables\nSteps: put pizza sauce on rice cakes, sprinkle pizza seasoning , basil and any optional vegetables on top of pizza, cover each pizza with a piece of fat free cheese, heat in microwave for 30-60 sec, depending on your preference\n\nDocument 2\nName: pizza con le patate potato pizza\nDescription: potato pizza\nIngredients: pizza dough, potatoes, rosemary, extra virgin olive oil, salt and pepper\nSteps: preheat oven to 500 f, evenly spread the potato slices over the pizza, top with the rosemary leaves, spread the olive oil on the pizza, add salt and pepper, bake the pizza for approximately 10 - 15 minutes, lift one side to check for readiness, pizza is ready when the bottom surface is light brown\n\nDocument 3\nName: make pizza\nDescription: pizza was originally started in naples, italy but has now become a popular type of food, it can go with almost anything and creating your own could taste better than the ones you buy in the shop. this article will give you 2 different ways of creating your pizza, one method is faster than the other.!\nIngredients: tomato paste, ham, cheese, olive, sausage\nSteps: preheat your oven to about 180 degrees, buy ready-made pizza bases , such as mccains ready-made pizza bases if you are short on time, if you have plenty of time , then you can just get dough, to make the method faster , dont get a huge pizza if youre only serving yourself or two, you dont want to take forever making your pizza, spread tomato paste on your pizza base, it is a good idea to buy one that is specially made for pizzas, you may want to sprinkle some italian seasoning with the sauce as well, make sure you dont put tomato base on the outside part of the pizza , or you wont have a crust for your pizza, dont make the sauce layer too thick or too thin, put a layer of ham onto your pizza, use nice , wholesome and delicious ham for your pizza to help it taste better, if a brand claims that it is the best ham , buy it , taste it , and see for yourself, in order to find the best ham , take a whole bunch of ham from the store, it doesnt have to be the perfect ham , just the ind of ham that you like, some people prefer smoked , while others prefer cooked ham , or honey ham, it all depends on you , and a couple of other people to help take votes, make sure that it is within its use-by date, ham is optional because some people dont like ham in their pizza, add a sprinkling of cheese, you can put as little or as much as you like , depending on your personal taste, mozzarella pizza cheese usually tastes really nice on pizzas, other people prefer other types of cheese, just like the ham , it is up to you, make sure that there are no spots without the cheese for your best taste, some people dont like the cheese , while some dont want too much cheese, common pizza takes the same old mozzarella , but if you are feeling creative , you can use any cheese you want-with any amount ! try being creative if you are artistic , and if you area serious chef then cook the traditional way, the benefits of creating your own is that you can find a better tasting way of pizza , and it is more fun !, enjoy it by adding sliced olives to your pizza but make sure that the olives are pitless, you dont have to have olives , this is optional, you can take almost any topping you want , like extra cheese , sausage , meatballs , pepperoni , chili peppers , supreme , veggies , and others, remember that it is your pizza and that you can do whatever you want with it, this time , youre the chef !, place your pizza on an oven tray sprayed with olive-oil , so that the pizza doesnt stick, you can use anything that prevents the pizza from sticking to the pan, this is important , and if you dont do that , you can end up without the pizza, put your pizza in the oven , and turn it down to about 160 degrees, too hot pizzas can burn you , and then youll have to wait to eat your pizza, you dont want to have to wait for the pizza to cool off, plus , you can get distracted and leave it in too long , then realize that you forgot the pizza was in the oven , and then you have charcoal in the oven, nobody wants their work to turn into burn ashes , dont they ? this happens with some people, take it out after about 15 - 25 minutes , depending on your oven, use your own judgment to tell when the pizza is ready, the cheese should be a golden brown , but it should not be burnt, grill method spread the sauce all over the tortilla but you might want to leave a little crust around the edge, leaving some around the edge will give you that delicious , crispy pizza taste, also , it will give you that nice crust to eat later after the main part of the pizza is eaten, put on your toppings and arrange, remember , you can take any topping you want , anywhere you want, just dont melt a pepperoni on a meatball !, grate the cheese over the pizza, you can use basically any cheese such as cheddar or edam, mozzarella cheese is the best for pizzas and it is the most common , but there are many cheeses to choose from, simply put it under the grill, be careful not to get burned and your pizza could sizzle and pop because of the common bubbles in the pizza crust, the bubbles form often in pizza , but the best pizza doesnt have bubbles, take it out after three minutes , it should be done ! now you can enjoy your tasty work, wood-fired oven method get your pizza base , it can be any sort , although it cant be more than 2 cms thick, place your tomato base and toppings on , do not be scared to put sausage , or any other raw meat on, it will cook, make sure your wood-fired oven is very hot for the best pizza ever and to help the pizza cook faster, put your pizza into the oven , preferably on a tray so the base does not burn, remember that you dont want your work to burn , and nobody wants a burnt pizza, turn your pizza every 30 seconds for two minutes , and at the 1:30 mark take it off the tray, now your pizza should be done, take your pizza out and enjoy !\n\nDocument 4\nName: margherita pizza white pizza\nDescription: tomato, basil, garlic, olive oil, mozzarella and ricotta cheese.\nIngredients: pizza crust, olive oil, pizza cheese, tomatoes, fresh basil, ricotta cheese, garlic powder\nSteps: brush pizza crust with olive oil, spread 2 cups of cheese evenly over pizza, place sliced tomato evenly over pizza, spread shredded basil evenly over pizza, place tablespoon size globs of ricotta evenly over pizza, sprinkle with garlic powder, spread remainig mozzarella over pizza, bake at 400 15 minute or until cheese is bubbly and brown on the edges\n\n\n\n## Instructions\n1. **Answer directly from the retrieved recipes when possible.** Use the information from the provided recipes to answer questions about ingredients, cooking methods, nutritional information, and preparation steps.\n\n2. **For ingredient questions:**\n   - Provide accurate amounts and measurements from the recipes\n   - Suggest possible substitutions based on common culinary knowledge\n   - Explain the purpose of key ingredients in the dish\n\n3. **For cooking technique questions:**\n   - Explain preparation methods mentioned in the recipes\n   - Clarify cooking times and temperatures\n   - Describe expected results and how to tell when food is properly cooked\n\n4. **For modification requests:**\n   - Suggest appropriate adjustments for dietary restrictions (vegan, gluten-free, etc.)\n   - Explain how to scale recipes up or down\n   - Offer ideas for flavor variations while maintaining the core identity of the dish\n\n5. **For general questions:**\n   - Provide brief culinary background/history when relevant\n   - Explain unfamiliar cooking terms\n   - Suggest pairings, serving suggestions, and storage recommendations\n\n## Response Format\n- Start with a direct answer to the user's question\n- Keep your responses concise but comprehensive\n- For multi-step instructions or complex concepts, organize information in a clear, logical structure\n- If the retrieved recipes don't contain sufficient information to answer the query, acknowledge the limitations and provide general culinary knowledge that might help\n- When suggesting modifications not explicitly in the retrieved recipes, clearly indicate these are your recommendations based on culinary principles\n\n## Limitations\n- Don't make claims about specific nutritional values unless they're mentioned in the retrieved recipes\n- If asked about topics completely unrelated to cooking or the recipes provided, politely redirect the conversation back to recipe-related topics\n- Don't invent or fabricate details about recipes that aren't in the retrieved data\n\n## User Query\nHow do you make pizza\n"


prompot_wihtout_documents = "\n\n# Recipe Assistant\n\n## Context\nYou are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.\n\n## Retrieved Recipes\nThe following recipes have been retrieved as most relevant to the user's query:\n\n\n\n## Instructions\n1. **Answer directly from the retrieved recipes when possible.** Use the information from the provided recipes to answer questions about ingredients, cooking methods, nutritional information, and preparation steps.\n\n2. **For ingredient questions:**\n   - Provide accurate amounts and measurements from the recipes\n   - Suggest possible substitutions based on common culinary knowledge\n   - Explain the purpose of key ingredients in the dish\n\n3. **For cooking technique questions:**\n   - Explain preparation methods mentioned in the recipes\n   - Clarify cooking times and temperatures\n   - Describe expected results and how to tell when food is properly cooked\n\n4. **For modification requests:**\n   - Suggest appropriate adjustments for dietary restrictions (vegan, gluten-free, etc.)\n   - Explain how to scale recipes up or down\n   - Offer ideas for flavor variations while maintaining the core identity of the dish\n\n5. **For general questions:**\n   - Provide brief culinary background/history when relevant\n   - Explain unfamiliar cooking terms\n   - Suggest pairings, serving suggestions, and storage recommendations\n\n## Response Format\n- Start with a direct answer to the user's question\n- Keep your responses concise but comprehensive\n- For multi-step instructions or complex concepts, organize information in a clear, logical structure\n- If the retrieved recipes don't contain sufficient information to answer the query, acknowledge the limitations and provide general culinary knowledge that might help\n- When suggesting modifications not explicitly in the retrieved recipes, clearly indicate these are your recommendations based on culinary principles\n\n## Limitations\n- Don't make claims about specific nutritional values unless they're mentioned in the retrieved recipes\n- If asked about topics completely unrelated to cooking or the recipes provided, politely redirect the conversation back to recipe-related topics\n- Don't invent or fabricate details about recipes that aren't in the retrieved data\n\n## User Query\nHow does Gordon Ramsay make his beef Wellington?\n"


prompt_ramsy_resoning = "\n\n# Recipe Assistant\n\n## Context\nYou are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.\n\n## Retrieved Recipes\nThe following recipes have been retrieved as most relevant to the user's query:\n\nDocument 0, Score: 0.2602\nName: pork wellington\nDescription: beef wellington is my absolute favorite dish, but when i went home for a visit, a dear friend made this for us (and he is the one who got me addicted to beef wellington) it is out of this world and great for company.\nIngredients: pork tenderloin, boursin cheese, olive oil, prosciutto, puff pastry, egg, water\nSteps: trim off all silverskin, butterfly making a 1 deep incision down the length of each and stuff each one with half of the cheese, roll prosciutto around each tenderloin, sear in oil in a large skillet over med-high heat until prosciutto is brown and crisp on all sides, chill thoroughly, wrap each tenderloin in puff pastry and transfer to a parchment-line baking sheet, brush wellingtons with egg wash and decorate with pastry cutouts if desired, cover with plastic and chill 1 hr, bake at 400f in lower third of oven for 30-35 minutes until golden brown, let rest 5 minutes before slicing\n\nDocument 1, Score: 0.2476\nName: individual beef scallop wellingtons rachael ray\nDescription: this is based on rachael rays recipe for wellingtons (well deserved, well dressed individual beef wellingtons), but ive also made them with sea scallops in addition to beef and they were astoundingly delicious.  :)  this is a *very* romantic dinner item.  *grin*  my husband was a big fan of this dish!!\nIngredients: extra virgin olive oil, butter, shallot, button mushroom, salt & freshly ground black pepper, garlic clove, dry sherry, filet mignon, pate, puff pastry, egg, water, fresh basil\nSteps: preheat oven to 400f, melt together the butter and oil over medium heat in a small skillet, add the shallot and mushrooms , season to taste with salt and pepper , and saut until mushrooms are browned , about 5 to 7 minutes , then add the garlic and saut for an additional 2 minutes, add the sherry , stir well , and saut until sherry evaporates, remove from heat, heat a nonstick skillet over high heat , drizzle with a little olive oil , and sear both sides of the pieces of beef and / or the scallops, set aside and keep warm, mix together the beaten egg and water to make an egg wash, on a lightly floured board , roll out each piece of the puff pastry until they\\re about 10x6 big, cut a thin strip , about 1 / 8th inch thick , from the 6-inch ends of the pastries and set aside for decorative purposes, on each piece of pastry , place half of the mushroom mixture , top with half of the pate , and then either 1 piece of seared beef or 2 seared scallops, brush the perimeter of the pastry with the egg wash, gather two opposite corners of the dough together , then the other two , and twist the 4 corners to seal, loosely tie the reserved strips around the top of each wellington, transfer wellingtons to a parchment or silpat lined baking sheet , then brush each wellington well with the egg wash and sprinkle with minced basil, bake at 400f for 15-20 minutes or until theyve turned a golden brown to your preference, remove from oven and let sit on baking sheet for 5 minutes, serve and enjoy !\n\nDocument 2, Score: 0.2356\nName: skin moist bake chicken\nDescription: i got this recipe from the back of the mayonnaise jar. it is very good and moist.\nIngredients: chicken parts, mayonnaise, italian breadcrumbs\nSteps: preheat oven 425 degress, take the skin off the chicken, brush chicken with mayo on both sides, place bread crumbs in a large food bag, add chicken one piece at a time tosschicken to coat, place chicken on a foiled- lined or non-stick baking sheet, bake chicken 30 minutes or untill golden and thoughly cooked\n\nDocument 3, Score: 0.2302\nName: chicken wellington mushroom veloute sauce\nDescription: this recipe is a little more involved but well worth the effort, in my opinion. this is a nice dish to serve when entertaining because it looks impressive. ive included a separate recipe for the mushroom sauce, which tops the chicken wellington. i really hope you enjoy this special recipe!\r\n*the prep time does not include 2 hours of refrigeration* \r\nnote: you can make the mushroom sauce earlier in the day and reheat while cooking dinner. it just cuts down on the amount of things you have to do at dinner time. ;)\nIngredients: boneless skinless chicken breasts, white wine, fresh basil, fresh oregano, puff pastry sheets, egg, water, boursin cheese, fresh parsley, chicken stock, unsalted butter, mushroom, shallots, flour, salt and pepper\nSteps: prep chicken:, in a mixing bowl , combine white wine , basil , oregano and mix well, place chicken breasts in a shallow dish and pour the wine mixture on top to marinate, refrigerate for 2 hours , turning the chicken breasts every half hour to insure all areas are marinated, prepare the mushroom veloute sauce:, in a saucepan , bring the chicken stock to a boil, in a saute pan , over medium heat , melt 2 tablespoons butter and saute mushrooms and shallots until tender, in another medium saucepan , over medium heat , melt the other 2 tablespoons of butter , slowly stir in flour to make a roux, cook for about 2 minutes , being careful not to burn, slowly add chicken stock and whisk until smooth, bring this sauce to a boil , then reduce heat to simmer, add mushrooms and shallots , salt and pepper to taste , and simmer for 10 minutes - then keep warm or reheat while the chicken is cooking, chicken wellington:, divide each boursin into thirds, remove chicken breasts from the marinade , reserving, put the marinade in a saute pan until the sauce gets hot , then add the chicken breasts and poach for 5-10 minutes , flipping halfway through, the timing really depends on how thick the breasts are, just make sure that the chicken is mostly cooked through, remove the chicken breasts and let cool for about 20 minutes, meanwhile , preheat the oven to 375 degrees, cut each puff pastry sheet in half, using 6 halves , place each breast in the center and top with a portion of boursin cheese, place each remaining puff pastry on top of eacj chicken breast , pinching the puff pastry to seal around the chicken, cut any excess puff pastry off, in a small bowl , make an egg wash by mixing the egg and water together , then brush the top of the wellington with egg wash, bake for 25 minutes, remove from oven and place each wellington in the center of the serving plate and top with mushroom veloute sauce, sprinkle with chopped parsley and serve\n\nDocument 4, Score: 0.2081\nName: salmon wellington\nDescription: a lucious spring entree!\nIngredients: salmon fillets, egg yolk, puff pastry, butter, salt & pepper, mushroom, onions, dry white wine, white vinegar, shallots, heavy cream, salt, white pepper, parsley\nSteps: make mushroom onion filling: melt butter or margarine in skillet and saute onion until lightly browned, add mushrooms and cook , stirring , until most of the liquid has evaporated, season to taste, remove to bowl and chill, fillingmay be refrigerated up to 2 days or may be frozen, roll half the pastry on a floured board into a rectangle approximately 14 inches long and 12 inches wide, cut into 4 rectangles, repeat with remaining dough, place salmon fillets on a greased baking sheet, tuck the thinner parts of fillet underneath , making them all the same thickness, divide mushroom onion filling among the fillets and spread evenly on top, cover each with a rectangle of pastry, tuck 1 / 2 inch of pastry under fillets, trim off excess dough, do not cover the entire bottom with pastry or it will become soggy, brush top and sides with egg yolk glaze , being careful not to let glaze drip, re-roll scraps of pastry and cut out small decorations, place on wellingtons and glaze the entire pastry again, maybe refrigerated up to 8 hours, make light white wine sauce: place wine , vinegar and shallots or onion in medium saucepan, bring to a boil , lower the heat , and simmer until slightly thickened and reduced to approximately 4 tablespoons, slowly whisk in the cream, simmer , stirring occasionally , until sauce thickens to desired consistency, this will take a little time , but as the water evaporates from the cream , it will thicken, season to taste with salt and pepper, stir in chives or parsley, reheat before serving, if sauce becomes too thick , thin down with additional wine or cream, serve hot, before serving , bring wellingtons to room temperature for 1 hour, preheat oven to 425f, bake wellingtons for 20-25 minutes or until the pastry is golden, at this time , if the fillets are 3 / 4 inch thick , they will be moist and flaky, spoon a small amount of sauce on each plate , place wellingtons on sauce and pass remaining sauce, the fine art of cooking involves personal choice, many preferences , ingredients , and procedures may not be consistent with what you know to be true, as with any recipe , you may find your personal intervention will be necessary, bon appetit !\n\n\n\n## Instructions\n1. **Answer directly from the retrieved recipes when possible.** Use the information from the provided recipes to answer questions about ingredients, cooking methods, nutritional information, and preparation steps.\n\n2. **For ingredient questions:**\n   - Provide accurate amounts and measurements from the recipes\n   - Suggest possible substitutions based on common culinary knowledge\n   - Explain the purpose of key ingredients in the dish\n\n3. **For cooking technique questions:**\n   - Explain preparation methods mentioned in the recipes\n   - Clarify cooking times and temperatures\n   - Describe expected results and how to tell when food is properly cooked\n\n4. **For modification requests:**\n   - Suggest appropriate adjustments for dietary restrictions (vegan, gluten-free, etc.)\n   - Explain how to scale recipes up or down\n   - Offer ideas for flavor variations while maintaining the core identity of the dish\n\n5. **For general questions:**\n   - Provide brief culinary background/history when relevant\n   - Explain unfamiliar cooking terms\n   - Suggest pairings, serving suggestions, and storage recommendations\n\n## Response Format\n- Start with a direct answer to the user's question\n- Keep your responses concise but comprehensive\n- For multi-step instructions or complex concepts, organize information in a clear, logical structure\n- If the retrieved recipes don't contain sufficient information to answer the query, acknowledge the limitations and provide general culinary knowledge that might help\n- When suggesting modifications not explicitly in the retrieved recipes, clearly indicate these are your recommendations based on culinary principles\n\n## Limitations\n- Don't make claims about specific nutritional values unless they're mentioned in the retrieved recipes\n- If asked about topics completely unrelated to cooking or the recipes provided, politely redirect the conversation back to recipe-related topics\n- Don't invent or fabricate details about recipes that aren't in the retrieved data\n\n## User Query\nWhat temperature should I bake chicken Wellington at?\n"


prompt_arab_dish_reasoning = "\n\n# Recipe Assistant\n\n## Context\nYou are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.\n\n## Retrieved Recipes\nThe following recipes have been retrieved as most relevant to the user's query:\n\nDocument 0, Score: 0.4509\nName: spicy pasta sauce\nDescription: a pasta sauce with a lot of kick!\nIngredients: spicy sausage, pasta sauce, rotel tomatoes & chilies, minced garlic cloves, parmesan cheese\nSteps: crumble sausage into skillet and cook until no longer pink, drain sausage well and wipe out skillet, return sausage to skillet and add pasta sauce , rotel , and garlic, simmer for 10 minutes, stir in parmesan cheese\n\nDocument 1, Score: 0.4362\nName: world easiest tomato soup\nDescription: this recipe came together one cold winter day when i was craving tomato soup and all i had in the pantry resembling tomatoes, was a jar of pasta sauce.\nIngredients: pasta sauce, heavy cream\nSteps: in a medium sized sauce pan add pasta sauce and heavy cream, simmer slowly over low heat for about 10 minutes or until heated through, i used members mark all-natural pasta sauce with four cheese from sams club , but im confident any brand pasta sauce with four cheese will turn out just as tasty\n\nDocument 2, Score: 0.3515\nName: romano casserole\nDescription: this is a really tasty, easy to put together casserole that your whole family will enjoy. the original recipe came from the label on a jar of pasta sauce. i use a smooth pasta sauce with tomato and romano cheese in it to make this but you could easily experiment with different types or use home-made.\nIngredients: raw rice, green capsicum, onion, pasta sauce, red kidney beans, lean ground beef, cheese, bacon\nSteps: place the rice in a greased casserole dish, cover the bottom evenly with the rice, sprinkle with the capsicum and onion, pour over half the pasta sauce, layer the kidney beans on top of the sauce and then spread the uncooked mince over the top of that, pour over the remaining pasta sauce and sprinkle with the bacon and cheese, bake , covered , at 180 degrees c for 1 hour, remove the lid for the last 10 minutes to brown the cheese\n\nDocument 3, Score: 0.3355\nName: baked chicken pasta sauce\nDescription: this ia a simple chicken dish with lots of flavor, use your own favorite pasta sauce for this, prego traditional pasta sauce works well with this --- for each extra pound chicken increase the sauce by 1 to 1-1/2 cups, remember to drain the fat completely before adding the sauce :)\nIngredients: chicken pieces, seasoning salt, ground black pepper, garlic powder, pasta sauce, spaghetti, parmesan cheese\nSteps: set oven to 375 degrees f, grease a 13 x 9-inch or a 2-quart baking dish, pat the chicken pieces dry using paper towels, season the chicken pieces with seasoned salt , black pepper and garlic power, place into the baking dish, bake for 30 minutes, drain any fat in the dish, pour the pasta sauce over the chicken then return to oven for another 25-30 minutes or until the chicken is cooked through, sprinkle parmesan cheese over, serve with cooked spaghetti\n\nDocument 4, Score: 0.3318\nName: cheesy vegetable pasta\nDescription: another great pasta recipe.  from taste of home.\nIngredients: olive oil, zucchini, vegetable oil, three-cheese pasta sauce, cooked spiral shaped pasta, mozzarella cheese\nSteps: heat oil in saucepan over medium heat, add zucchini and cook until crisp-tender, add pasta sauce and pasta, heat through, stir in the cheese\n\n\n\n## Instructions\n1. **Answer directly from the retrieved recipes when possible.** Use the information from the provided recipes to answer questions about ingredients, cooking methods, nutritional information, and preparation steps.\n\n2. **For ingredient questions:**\n   - Provide accurate amounts and measurements from the recipes\n   - Suggest possible substitutions based on common culinary knowledge\n   - Explain the purpose of key ingredients in the dish\n\n3. **For cooking technique questions:**\n   - Explain preparation methods mentioned in the recipes\n   - Clarify cooking times and temperatures\n   - Describe expected results and how to tell when food is properly cooked\n\n4. **For modification requests:**\n   - Suggest appropriate adjustments for dietary restrictions (vegan, gluten-free, etc.)\n   - Explain how to scale recipes up or down\n   - Offer ideas for flavor variations while maintaining the core identity of the dish\n\n5. **For general questions:**\n   - Provide brief culinary background/history when relevant\n   - Explain unfamiliar cooking terms\n   - Suggest pairings, serving suggestions, and storage recommendations\n\n## Response Format\n- Start with a direct answer to the user's question\n- Keep your responses concise but comprehensive\n- For multi-step instructions or complex concepts, organize information in a clear, logical structure\n- If the retrieved recipes don't contain sufficient information to answer the query, acknowledge the limitations and provide general culinary knowledge that might help\n- When suggesting modifications not explicitly in the retrieved recipes, clearly indicate these are your recommendations based on culinary principles\n\n## Limitations\n- Don't make claims about specific nutritional values unless they're mentioned in the retrieved recipes\n- If asked about topics completely unrelated to cooking or the recipes provided, politely redirect the conversation back to recipe-related topics\n- Don't invent or fabricate details about recipes that aren't in the retrieved data\n\n## User Query\nI'm looking for a spicy pasta sauce, but I forgot the name. I think it was related to Arabs?\n"


prompt_rhode_reasoning = "\n\n# Recipe Assistant\n\n## Context\nYou are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.\n\n## Retrieved Recipes\nThe following recipes have been retrieved as most relevant to the user's query:\n\nDocument 0, Score: 0.3569\nName: rhode island coffee milk\nDescription: this is a popular comfort food in rhode island. in fact, it is the official state drink of rhode island. it comes from the book ill have what theyre having- legendary local cuisine page 206\nIngredients: coffee-flavored syrup, milk\nSteps: in a large glass , add coffee syrup and milk, stir well, taste and add more syrup if you want a stronger coffee taste\n\nDocument 1, Score: 0.2710\nName: rhode island johnnycakes\nDescription: these are a rhode island tradition. serve hot with maple syrup!\nIngredients: flour, cornmeal, baking powder, salt, eggs, buttermilk, butter\nSteps: mix all dry ingredients, add eggs to buttermilk in a bowl and beat lightly, add the buttermilk to the dry ingredients and blend, fold the melted butter into the batter, let batter rest for 20 minutes and adjust consistency with a few tablespoons of water if necessary, preheat a flat griddle to 350f, use 1 1 / 2 oz, of batter per cake and lightly grease the griddle, pour the batter and cook until it bubbles and is golden brown , then flip over for 30 seconds or until browned\n\nDocument 2, Score: 0.2378\nName: rhode island clam cakes\nDescription: clam cakes are a summer tradition in rhode island.  a ride to the shore will always necessitate a stop at georges or aunt carries for a dozen clam cakes and a pint of chowder!\nIngredients: all-purpose flour, salt, pepper, baking powder, eggs, milk, minced clams, dried onion flakes, oil\nSteps: mix all ingredients together, drop by ice cream scoop full in hot oil and fry until golden brown, drain on paper towels, salt as desired and serve hot\n\n\n\n## Instructions\n1. **Answer directly from the retrieved recipes when possible.** Use the information from the provided recipes to answer questions about ingredients, cooking methods, nutritional information, and preparation steps.\n\n2. **For ingredient questions:**\n   - Provide accurate amounts and measurements from the recipes\n   - Suggest possible substitutions based on common culinary knowledge\n   - Explain the purpose of key ingredients in the dish\n\n3. **For cooking technique questions:**\n   - Explain preparation methods mentioned in the recipes\n   - Clarify cooking times and temperatures\n   - Describe expected results and how to tell when food is properly cooked\n\n4. **For modification requests:**\n   - Suggest appropriate adjustments for dietary restrictions (vegan, gluten-free, etc.)\n   - Explain how to scale recipes up or down\n   - Offer ideas for flavor variations while maintaining the core identity of the dish\n\n5. **For general questions:**\n   - Provide brief culinary background/history when relevant\n   - Explain unfamiliar cooking terms\n   - Suggest pairings, serving suggestions, and storage recommendations\n\n## Response Format\n- Start with a direct answer to the user's question\n- Keep your responses concise but comprehensive\n- For multi-step instructions or complex concepts, organize information in a clear, logical structure\n- If the retrieved recipes don't contain sufficient information to answer the query, acknowledge the limitations and provide general culinary knowledge that might help\n- When suggesting modifications not explicitly in the retrieved recipes, clearly indicate these are your recommendations based on culinary principles\n\n## Limitations\n- Don't make claims about specific nutritional values unless they're mentioned in the retrieved recipes\n- If asked about topics completely unrelated to cooking or the recipes provided, politely redirect the conversation back to recipe-related topics\n- Don't invent or fabricate details about recipes that aren't in the retrieved data\n\n## User Query\nWhat's a typical food item from Rhode Island?\n"


prompt_calamari_reasoning = "\n\n# Recipe Assistant\n\n## Context\nYou are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.\n\n## Retrieved Recipes\nThe following recipes have been retrieved as most relevant to the user's query:\n\nDocument 0, Score: 0.2734\nName: bill sunday night pasta\nDescription: i put this together late on a sunday night and it was so good, i just had to share it with you. i ate it along with a glass of red wine, but if eaten with a salad, it would be a nice solo lunch.\nIngredients: dry pasta, tomatoes, pepperoni, salt, garlic, olive oil, parmesan cheese\nSteps: first , dice the tomato and sprinkle with the salt, start 2 quarts of salted water to boil , and when it is a rolling boil , add the pasta, cook until desired doneness dice the garlic and put in the bottom of a pasta dish with one tablespoon of olive , while the pasta is cooking, add the hot cooked pasta to the dish , and toss until the the olive oil and garlic coats the pasta, add pepperoini and tomato and toss again, drizzle the rest of the olive oil on top , and add parmesan cheese\n\nDocument 1, Score: 0.2712\nName: calamari special\nDescription: this is one of the nicest and best tasting calamari recipes that i have eaten. i find that the calamari is not tough as i have experienced in many restaurants. a friend chef and i developed this marvelous recipe that is so easy to prepare.\nIngredients: calamari, whole milk, all-purpose flour, panko breadcrumbs, vegetable oil, garlic cloves, sambal oelek\nSteps: firstly , prepare a deep-fryer with a good vegetable oil and heat to 375 degrees f , or use a medium size deep frying pan on high heat, defrost calamari and rinse under cold running water, cut calamari into 1 / 4 inch strips, if using fresh calamari , just cut into 1 / 4 inch strips, pat dry calamari with paper towels, in a medium size mixing bowl , mix together flour and panko bread crumbs, add milk to a bowl and dip calamari in the milk, toss calamari strips in flour / crumb mixture just to coat, you do not want to have a heavy coating as it could drop off when frying, flash fry coated calamari in prepared deep-fryer or frying pan for about 15 seconds , deep frying in small amounts, in a frying pan , heat 1 / 2 cup of vegetable oil to medium-high heat, add garlic and sambal oelek and stir well , cover with a screen to prevent oil from spattering or splashing, saut flash-fried calamari for about 1 minute or until nicely light brown, do not over fry as it could get chewy, serve calamari with your favorite dip\n\nDocument 2, Score: 0.2688\nName: crumbed calamari\nDescription: i love calamari. this is how to crumb them at home. look for more calamari recipes coming soon. serve with tartar sauce and enjoy the experience\nIngredients: calamari, breadcrumbs, salt, pepper, garlic powder, egg, milk, oil, tartar sauce\nSteps: mix the breadcrumbs with the salt , pepper and garlic powder and place into a plastic bag so you can shake the crumbs onto the calamari, beat the egg and combine with the milk and place this into a small dish, heat the oil up to 160c degrees, dip the calamari rings individually into the egg / milk mix and then into the plastic bag and shake, the calamari should be well coated, deep fry until a nice brown tinge, i tend to fry mine for about 4-5 minutes, it also depends on the calamari some it is better at a lower heat for longer, this rule especially applies to bought crumbed calamari, enjoy, serve with tartar sauce\n\nDocument 3, Score: 0.2567\nName: prawn chicken stuffed calamari thai style\nDescription: this stunning dish comes from melbournes wonderful royal thai restaurant palm sugar and was published in the age newspaper some time ago.  it looks impressive and is highlighted by the tasty chilli capsicum sauce.\nIngredients: calamari, prawns, minced chicken, garlic clove, fish sauce, frypowder, oil, red capsicum, vinegar, sugar, fresh chili peppers, coriander leaves\nSteps: calamari:, mix the prawn chicken and garlic together, stuff the calamari with the mixture until full , but taking care not to over-stuff as the calamari will expand when cooked, secure the open end of the calamari with toothpick / s, in a steamer , steam the calamari for about 25- 30 minutes, use this opportunity to make sauce, when the steaming is completed , roll the calamari tube in a shallow dish containing the fry powder, once coated , remove the calamari tube and add water to the fry powder to make a paste as per the packets instructions, heat oil is a wok so that it is just starting to smoke, add calamari tube and fry quickly on all sides the coating should be a light , crisp batter, remove calamari from oil , and cut into slices / discs, place the discs on a plate , pour / drizzle sauce over the top and sprinkle coriander leaves over the dish, sauce:, add oil to small saucepan and saut capsicum , garlic and chilli until the capsicum is soft, add the fish sauce , vinegar and sugar to the saucepan, cook until sauce starts to thicken , being careful not to burn\n\nDocument 4, Score: 0.2543\nName: calamari tomato sauce\nDescription: i love calamari no matter how its cooked, but this is a good one!\nIngredients: calamari, oil, garlic cloves, crushed red pepper flakes, tomato sauce, parsley, salt, red wine\nSteps: cut calamari into small strips, place in saucepan and cook over medium heat for 15 minutes drain all but 1 cup of liquid, add oil , garlic , chili pepper , sauce , parsley and salt, cover and simmer for 25 minutes, add wine and cook for an additional 5 minutes, adjust seasonings to taste, serve over cooked rice or noodles\n\n\n\n## Instructions\n1. **Answer directly from the retrieved recipes when possible.** Use the information from the provided recipes to answer questions about ingredients, cooking methods, nutritional information, and preparation steps.\n\n2. **For ingredient questions:**\n   - Provide accurate amounts and measurements from the recipes\n   - Suggest possible substitutions based on common culinary knowledge\n   - Explain the purpose of key ingredients in the dish\n\n3. **For cooking technique questions:**\n   - Explain preparation methods mentioned in the recipes\n   - Clarify cooking times and temperatures\n   - Describe expected results and how to tell when food is properly cooked\n\n4. **For modification requests:**\n   - Suggest appropriate adjustments for dietary restrictions (vegan, gluten-free, etc.)\n   - Explain how to scale recipes up or down\n   - Offer ideas for flavor variations while maintaining the core identity of the dish\n\n5. **For general questions:**\n   - Provide brief culinary background/history when relevant\n   - Explain unfamiliar cooking terms\n   - Suggest pairings, serving suggestions, and storage recommendations\n\n## Response Format\n- Start with a direct answer to the user's question\n- Keep your responses concise but comprehensive\n- For multi-step instructions or complex concepts, organize information in a clear, logical structure\n- If the retrieved recipes don't contain sufficient information to answer the query, acknowledge the limitations and provide general culinary knowledge that might help\n- When suggesting modifications not explicitly in the retrieved recipes, clearly indicate these are your recommendations based on culinary principles\n\n## Limitations\n- Don't make claims about specific nutritional values unless they're mentioned in the retrieved recipes\n- If asked about topics completely unrelated to cooking or the recipes provided, politely redirect the conversation back to recipe-related topics\n- Don't invent or fabricate details about recipes that aren't in the retrieved data\n\n## User Query\nGive me some ideas for pasta dishes that contain calamari.\n"


prompt_cactus_reasoning = "\n\n# Recipe Assistant\n\n## Context\nYou are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.\n\n## Retrieved Recipes\nThe following recipes have been retrieved as most relevant to the user's query:\n\nDocument 0, Score: 0.3095\nName: california cactus salad\nDescription: this is a latin inspired recipe adapted from cooking light magazine. i sometimes can find cactus pads in walmart.\nIngredients: cactus pieces, olive oil, garlic cloves, canned black beans, green onions, fresh cilantro, fresh lime juice, sugar, ground cumin, dried chipotle powder, tomatoes, lettuce, queso fresco, sour cream\nSteps: peel thorns from cactus pads with a vegetable peeler , leaving the green skin, cut cactus pads into thin strips, heat the oil in a large nonstick skillet over medium high heat, add the cactus and garlic , saute 7-8 minutes or until cactus is tender, combine the cactus mixture , beans and the next 7 ingredients, arrange 1 1 / 2 cups lettuce on each of 4 plates , top each with 1 / 2 cup cactus mixture , 1 1 / 2 tbls, cheese , and about 2 teaspoons sour cream, enjoy !, yield: 4 servings\n\nDocument 1, Score: 0.3089\nName: cactus mexican style\nDescription: this is so good for my family. family favorite. healthy and yummy!\nIngredients: cactus pieces, tomatoes, white onion, jalapeno pepper, cilantro, monterey jack cheese, salt, corn oil\nSteps: steam the cactus until softened, drain the cactus, fry all the ingredients except for the monterrey cheese, combine fried ingredients with the cactus and allow to simmer for 15 minutes, top with cheese before serving\n\nDocument 2, Score: 0.2848\nName: cactus salad\nDescription: ive never eaten cactus before but this does not sound to bad.\nIngredients: cactus pieces, water, salt, scallions, cider vinegar, lime juice, sun-dried tomato packed in oil, oregano leaves, red onion, queso fresco, chipotle chiles\nSteps: pare off edges of cactus paddles and slice into inch strips, bring water , salt , and scallions to a boil, drop in cactus strips and bring to boiling, reduce heat and simmer until tender, drain and rinse, combine vinegar , lime juice , oil , sun dried tomatoes , and oregano in separate bowl, arrange cactus strips and sauted onions on platter , pour vinegar mixture over salad , sprinkle cheese and chilies on top and serve\n\nDocument 3, Score: 0.2654\nName: cactus candy\nDescription: from allrecipes, a taste of the desert!\nIngredients: prickly pear cactus, granulated sugar, water, orange juice\nSteps: select prickly pear cactus, remove spines and outside layer with large knife, i found this at walmart !, cut pulp across in slices one-inch thick, soak overnight in cold water, remove from water , cut in 1 cubes and cook in boiling water til tender, drain, heat the sugar and water until sugar is dissolved, then add cactus, cook slowly in the syrup until nearly all the syrup is absorbed, do not scorch ! remove cactus from syrup , drain and roll in granulated or powdered sugar, for colored cactus candy , any vegetable food coloring may be added to the syrup\n\nDocument 4, Score: 0.2645\nName: pico de gallo cactus\nDescription: this is a recipe from my grandmother. there isnt much difference in pico de gallo recipes but this one adds a great flavor when you add the cactus. the cactus can be located in the mexican food isle. my friends really enjoy the recipe. i hope you enjoy.\nIngredients: tomatoes, white onion, cilantro leaf, jalapeno peppers, cactus pieces\nSteps: mix all together, add salt and garlic salt to taste\n\n\n\n## Instructions\n1. **Answer directly from the retrieved recipes when possible.** Use the information from the provided recipes to answer questions about ingredients, cooking methods, nutritional information, and preparation steps.\n\n2. **For ingredient questions:**\n   - Provide accurate amounts and measurements from the recipes\n   - Suggest possible substitutions based on common culinary knowledge\n   - Explain the purpose of key ingredients in the dish\n\n3. **For cooking technique questions:**\n   - Explain preparation methods mentioned in the recipes\n   - Clarify cooking times and temperatures\n   - Describe expected results and how to tell when food is properly cooked\n\n4. **For modification requests:**\n   - Suggest appropriate adjustments for dietary restrictions (vegan, gluten-free, etc.)\n   - Explain how to scale recipes up or down\n   - Offer ideas for flavor variations while maintaining the core identity of the dish\n\n5. **For general questions:**\n   - Provide brief culinary background/history when relevant\n   - Explain unfamiliar cooking terms\n   - Suggest pairings, serving suggestions, and storage recommendations\n\n## Response Format\n- Start with a direct answer to the user's question\n- Keep your responses concise but comprehensive\n- For multi-step instructions or complex concepts, organize information in a clear, logical structure\n- If the retrieved recipes don't contain sufficient information to answer the query, acknowledge the limitations and provide general culinary knowledge that might help\n- When suggesting modifications not explicitly in the retrieved recipes, clearly indicate these are your recommendations based on culinary principles\n\n## Limitations\n- Don't make claims about specific nutritional values unless they're mentioned in the retrieved recipes\n- If asked about topics completely unrelated to cooking or the recipes provided, politely redirect the conversation back to recipe-related topics\n- Don't invent or fabricate details about recipes that aren't in the retrieved data\n\n## User Query\nI want to eat something with cactus. How many recipes do you know?\n"


propmt_borscht_reasoning = "\n\n# Recipe Assistant\n\n## Context\nYou are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.\n\n## Retrieved Recipes\nThe following recipes have been retrieved as most relevant to the user's query:\n\nDocument 0, Score: 0.3533\nName: uncle bill russian doukhobour borscht\nDescription: this russian/doukhobour borscht originated with my russian grandmother who was a vegetarian. traditionally, the russian/doukhobours used about 1 pound of butter and at least 3 cups of whipping cream. i finally was able to make numerous adjustments to the ingredients and then reduce the amount of butter and whipping cream so that the borscht tastes just as good, but not as rich. the procedure is rather lengthy, but is worth every bit of that time. the borscht may be canned or frozen in containers.\r\nthe original russian borscht had meat in it and also they used many red beets.\nIngredients: water, canned tomatoes, potatoes, carrots, onions, salt, red beets, butter, whipping cream, green cabbage, cauliflower florets, celery ribs, green pepper, sweet red pepper, fresh ground black pepper, dried dill weed, fresh lemon juice, ground cayenne pepper\nSteps: in a large stock pot , add water and bring to boil, add 1 / 2 cup of mashed tomatoes , quartered potatoes , chopped carrots , chopped onions , salt and beets and return to boil, reduce heat to medium and cook uncovered for 15 minutes or until potatoes are just cooked, meanwhile , begin to prepare all vegetables, when potatoes are cooked , remove with a slotted spoon into a mixing bowl, add 3 tablespoons butter to potatoes , mash well, stir in whipping cream , mixing well to incorporate, set aside, in a large frying pan , melt 3 tablespoons butter , add chopped onions and saute until just softened , about 5 minutes, add remaining mashed tomatoes and simmer uncovered until reduced to a creamy consistency, the sauce will be thickened, cover , reduce heat to keep warm, in another large frying pan , melt the remainder 3 tablespoons butter , add 4 cups of shredded cabbage and on medium heat , fry until softened and very lightly browned , about 15 minutes , do not burn, remove from heat and set aside, to the cooking pot , add cubed potatoes and 1 / 2 cup of tomato-onion mixture, cover and cook for 10 minutes, add cauliflower and celery and continue to boil gently until potatoes are just tender, stir in remaining 4 cups of raw shredded cabbage and return soup to boil, reduce heat, stir in mashed potatoes , fried cabbage and remaining tomato-onion mixture, add diced green and red peppers , freshly ground black pepper , dill weed , lemon juice, return to boil and cook for 3 more minutes, cover and reduce heat to just warm and let borscht sit for 30 minutes for flavors to blend, remove beets and discard or eat them separately, the beets are for coloring only, adjust seasonings to taste, if borscht is too sweet to your liking , stir in some additional lemon juice, if desired , serve with a sprinkle of cayenne pepper, refrigerate any unused portions, traditionally , medium or sharp cheddar cheese pieces and fresh bread or buns are served with the borscht\n\nDocument 1, Score: 0.3340\nName: beautiful borscht salmon\nDescription: plate up a palette of pretty colours. rich orange grilled salmon, raspberry red borscht sauce, bright white crème fraiche, deep green dill sprigs. a dish that will delight your taste buds as well as your eyes. lovely served with fresh steamed haricots verts and lemon rice. the recipe makes 8 servings. but do not worry if you are cooking for 2 or 4. any unused borscht sauce is easily thinned to soup consistency and very flavourful the next day. enjoy your borscht again hot or as a beautiful cold summer soup!\nIngredients: beet, olive oil, shallot, carrot, celery, napa cabbage, garlic clove, canned broth, bay leaf, peppercorns, salt, fresh dill, red wine vinegar, creme fraiche, salmon fillets, garlic salt, cracked black pepper\nSteps: step 1 is good to do in advance, place beet in a pot of boiling water and cook uncovered 15 minutes until semi-tender, remove from heat , cover pot , and allow beet to steam another 15 minutes, remove beet from hot water with slotted spoon, once cool , peel and dice beet, steps 2 and 3 may also be done in advance, saute aromatic shallots , carrot , and celery in olive oil until semi-tender, stir in cabbage and garlic, add diced beet , broth , bay leaf , and peppercorns, adjust heat , cover pot and gently simmer for one half hour until vegetables are tender, remove from heat, discard bay leaf and peppercorns, add salt , dill , and red wine vinegar, puree sauce in blender or food processor until smooth, stir in crme fraiche, taste and adjust seasonings as desired, borscht sauce may be served hot or cold, keep heated in saucepan until serving time, or chill sauce in refrigerator, while borscht sauce is cooking , rub salmon fillets on both sides with olive oil, sprinkle salmon with garlic salt and pepper, grill fillets 4-5 minutes per side until done, to serve , plate each salmon fillet , then top with 1 / 4 cup borscht sauce , additional crme fraiche , and sprigs of fresh dill, a beautiful presentation your dinner guests will not soon forget !, this recipe makes just over 2 cups borscht sauce, canned broth may be used to thin any unused sauce to preferred soup consistency, stir in broth and reheat , or enjoy a cold summer soup, very delicious the next day !\n\nDocument 2, Score: 0.3141\nName: borscht ii\nDescription: borscht is like chili; theres no one recipe for it.  this recipe is somewhat unusual because it does not have cabbage; the addition of lime juice and sugar may also not be traditional.  my mother did get it from a russian coworker.  it is the borscht i grew up with and i love it.\nIngredients: beef stew meat, chicken stock, beets, onion, tomatoes, lime, juice of, sugar, salt and pepper, sour cream\nSteps: combine the first five ingredients in a large non-reactive pot, bring to a boil , then reduce heat and allow the soup to simmer for 2= hours, half an hour before serving , remove beets , keeping the broth at a simmer, when beets are cool enough to handle , peel and grate them, then return them to the pot, stir in lime juice and sugar , then season with salt and pepper, serve , garnished with a dollop of sour cream and / or some snipped chives\n\nDocument 3, Score: 0.3123\nName: authentic russian ukrainian borscht\nDescription: this is the real way to make borscht. no canned  veggies, no messing around. just real hearty borscht, the way my mother makes it. good ingredients are key, especially the meat, which should be fresh and thick. once youve had this borscht, you will not be able to eat any other. for the non-kosher, try it au-natural, or with a nice spoonful of sour cream mixed in (please, not low fat, enjoy this!).\nIngredients: beef, beets, lemons, head of cabbage, carrots, tomato sauce, dill, instant chicken-style consomme soup and seasoning mix, onion, potatoes\nSteps: to a large pot , add the meat and enough water to fill up the pot to halfway, remove all froth that comes up from the meat, continue to boil on med-high until no more froth comes up, reduce heat to low and add consomme mix , whole onion , dill , beats , cabbage , and tomato sauce, fill with water , cover partially with lid , and let cook for 90min, after that hour and a half of cooking , make sure meat is soft - then add the carrots , and squeeze all the juice you can possible juice from the lemons into the soup, you can add the rice and / or potatoes at this point , if you like, cook , partially covered , for another 30-45 minutes , until making sure the potatoes and carrots are tender , and the rice is ready, the soup is great on day 1 , but tastes much better on days 2-3, enjoy the soup while its very hot\n\nDocument 4, Score: 0.3065\nName: simple borscht\nDescription: this is a simple and easy recipe for borscht, from classic conran, plain, simple and satisfying food.\r\nthere are many good borscht recipes, this recipe creates a thinner but very flavourful soup which is sweet but still has that great earthy flavour. this is a great soup both hot and cold.\r\ngarnish with a swirl of soured cream or plain yogurt and some chopped chives\nIngredients: butter, beetroot, onion, carrot, garlic cloves, caster sugar, beef stock, salt and pepper, lemon, juice of\nSteps: melt butter in a large pan over a gentle heat and slowly sweat the beet root , onion , carrot and garlic , turning the vegetables over in the butter, add the sugar and stock to the pan , season with a few grinds of pepper , bring the soup to a simmer and cook for about 40 minutes until the vegetables are tender, using a blender , blend the soup until it is entirely smooth , then add the lemon juice and salt to taste\n\n\n\n## Instructions\n1. **Answer directly from the retrieved recipes when possible.** Use the information from the provided recipes to answer questions about ingredients, cooking methods, nutritional information, and preparation steps.\n\n2. **For ingredient questions:**\n   - Provide accurate amounts and measurements from the recipes\n   - Suggest possible substitutions based on common culinary knowledge\n   - Explain the purpose of key ingredients in the dish\n\n3. **For cooking technique questions:**\n   - Explain preparation methods mentioned in the recipes\n   - Clarify cooking times and temperatures\n   - Describe expected results and how to tell when food is properly cooked\n\n4. **For modification requests:**\n   - Suggest appropriate adjustments for dietary restrictions (vegan, gluten-free, etc.)\n   - Explain how to scale recipes up or down\n   - Offer ideas for flavor variations while maintaining the core identity of the dish\n\n5. **For general questions:**\n   - Provide brief culinary background/history when relevant\n   - Explain unfamiliar cooking terms\n   - Suggest pairings, serving suggestions, and storage recommendations\n\n## Response Format\n- Start with a direct answer to the user's question\n- Keep your responses concise but comprehensive\n- For multi-step instructions or complex concepts, organize information in a clear, logical structure\n- If the retrieved recipes don't contain sufficient information to answer the query, acknowledge the limitations and provide general culinary knowledge that might help\n- When suggesting modifications not explicitly in the retrieved recipes, clearly indicate these are your recommendations based on culinary principles\n\n## Limitations\n- Don't make claims about specific nutritional values unless they're mentioned in the retrieved recipes\n- If asked about topics completely unrelated to cooking or the recipes provided, politely redirect the conversation back to recipe-related topics\n- Don't invent or fabricate details about recipes that aren't in the retrieved data\n\n## User Query\nWhat is the most important ingredient in borscht?\n"


propt_cactus_bad_prompt = "\nYou are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.\n\nThe following recipes have been retrieved as most relevant to the user's query:\nDocument 0, Score: 0.3095\nName: california cactus salad\nDescription: this is a latin inspired recipe adapted from cooking light magazine. i sometimes can find cactus pads in walmart.\nIngredients: cactus pieces, olive oil, garlic cloves, canned black beans, green onions, fresh cilantro, fresh lime juice, sugar, ground cumin, dried chipotle powder, tomatoes, lettuce, queso fresco, sour cream\nSteps: peel thorns from cactus pads with a vegetable peeler , leaving the green skin, cut cactus pads into thin strips, heat the oil in a large nonstick skillet over medium high heat, add the cactus and garlic , saute 7-8 minutes or until cactus is tender, combine the cactus mixture , beans and the next 7 ingredients, arrange 1 1 / 2 cups lettuce on each of 4 plates , top each with 1 / 2 cup cactus mixture , 1 1 / 2 tbls, cheese , and about 2 teaspoons sour cream, enjoy !, yield: 4 servings\n\nDocument 1, Score: 0.3089\nName: cactus mexican style\nDescription: this is so good for my family. family favorite. healthy and yummy!\nIngredients: cactus pieces, tomatoes, white onion, jalapeno pepper, cilantro, monterey jack cheese, salt, corn oil\nSteps: steam the cactus until softened, drain the cactus, fry all the ingredients except for the monterrey cheese, combine fried ingredients with the cactus and allow to simmer for 15 minutes, top with cheese before serving\n\nDocument 2, Score: 0.2848\nName: cactus salad\nDescription: ive never eaten cactus before but this does not sound to bad.\nIngredients: cactus pieces, water, salt, scallions, cider vinegar, lime juice, sun-dried tomato packed in oil, oregano leaves, red onion, queso fresco, chipotle chiles\nSteps: pare off edges of cactus paddles and slice into inch strips, bring water , salt , and scallions to a boil, drop in cactus strips and bring to boiling, reduce heat and simmer until tender, drain and rinse, combine vinegar , lime juice , oil , sun dried tomatoes , and oregano in separate bowl, arrange cactus strips and sauted onions on platter , pour vinegar mixture over salad , sprinkle cheese and chilies on top and serve\n\nDocument 3, Score: 0.2654\nName: cactus candy\nDescription: from allrecipes, a taste of the desert!\nIngredients: prickly pear cactus, granulated sugar, water, orange juice\nSteps: select prickly pear cactus, remove spines and outside layer with large knife, i found this at walmart !, cut pulp across in slices one-inch thick, soak overnight in cold water, remove from water , cut in 1 cubes and cook in boiling water til tender, drain, heat the sugar and water until sugar is dissolved, then add cactus, cook slowly in the syrup until nearly all the syrup is absorbed, do not scorch ! remove cactus from syrup , drain and roll in granulated or powdered sugar, for colored cactus candy , any vegetable food coloring may be added to the syrup\n\nDocument 4, Score: 0.2645\nName: pico de gallo cactus\nDescription: this is a recipe from my grandmother. there isnt much difference in pico de gallo recipes but this one adds a great flavor when you add the cactus. the cactus can be located in the mexican food isle. my friends really enjoy the recipe. i hope you enjoy.\nIngredients: tomatoes, white onion, cilantro leaf, jalapeno peppers, cactus pieces\nSteps: mix all together, add salt and garlic salt to taste\n\n\n\n## User Query\nI want to eat something with cactus. How many recipes do you know?\n"


In [7]:
def experiment_run_all_prompt_generation():
    print("#########################")
    print("Running all prompt generation experiments")
    print("#########################")

    prompts = {
        "propt_cactus_bad_prompt": propt_cactus_bad_prompt,
        "propmt_borscht_reasoning": propmt_borscht_reasoning,
        "prompt_cactus_reasoning": prompt_cactus_reasoning,
        "prompt_calamari_reasoning": prompt_calamari_reasoning,
        "prompt_rhode_reasoning": prompt_rhode_reasoning,
        "prompt_arab_dish_reasoning": prompt_arab_dish_reasoning,
        "prompt_ramsy_resoning": prompt_ramsy_resoning,
        "prompt_with_scores": prompt_with_scores,
        "prompt_without_scores": prompt_without_scores,
        "prompot_wihtout_documents": prompot_wihtout_documents,
    }

    for prompt_name, prompt in prompts.items():
        response = generate_response(prompt)
        print(response)
        with open(f"prompts_new/{prompt_name}.txt", "w") as f:
            f.write(response)



In [8]:
# import torch
# import transformers
# from huggingface_hub import login
# from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model, tokenizer = None, None


def get_model_and_tokenizer():
    global model, tokenizer
    if model is None or tokenizer is None:
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            trust_remote_code=True,
            quantization_config=bnb_config,
            device_map="cuda",
        )
        tokenizer = AutoTokenizer.from_pretrained(model_id)
    return model, tokenizer


def generate_response(prompt):
    model, tokenizer = get_model_and_tokenizer()

    encoded_prompt = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}], return_tensors="pt"
    )
    encoded_prompt = encoded_prompt.to('cuda')
    generated_ids = model.generate(encoded_prompt, max_new_tokens=1000, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    return decoded[0]

experiment_run_all_prompt_generation()


#########################
Running all prompt generation experiments
#########################


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<s> [INST] 
You are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.

The following recipes have been retrieved as most relevant to the user's query:
Document 0, Score: 0.3095
Name: california cactus salad
Description: this is a latin inspired recipe adapted from cooking light magazine. i sometimes can find cactus pads in walmart.
Ingredients: cactus pieces, olive oil, garlic cloves, canned black beans, green onions, fresh cilantro, fresh lime juice, sugar, ground cumin, dried chipotle powder, tomatoes, lettuce, queso fresco, sour cream
Steps: peel thorns from cactus pads with a vegetable peeler , leaving the green skin, cut cactus pads into thin strips, heat the oil in a large nonstick skillet over medium

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

<s> [INST] 

# Recipe Assistant

## Context
You are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.

## Retrieved Recipes
The following recipes have been retrieved as most relevant to the user's query:

Document 0, Score: 0.2602
Name: pork wellington
Description: beef wellington is my absolute favorite dish, but when i went home for a visit, a dear friend made this for us (and he is the one who got me addicted to beef wellington) it is out of this world and great for company.
Ingredients: pork tenderloin, boursin cheese, olive oil, prosciutto, puff pastry, egg, water
Steps: trim off all silverskin, butterfly making a 1 deep incision down the length of each and stuff each one with half of the cheese, roll 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<s> [INST] 

# Recipe Assistant

## Context
You are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.

## Retrieved Recipes
The following recipes have been retrieved as most relevant to the user's query:

Document 0, Score: 0.5509
Name: semi copycat sporky pizza bertucci
Description: this was my favorite pizza when i lived in boston.  ive tried to get our local pizza places to make and sell it, but to no avail. they just make the same old - same old pizza!
i have not included oven temp or time to cook because it all depends on the pizza you buy.
Ingredients: frozen cheese pizza, tomatoes, ricotta cheese, mozzarella cheese, olive oil, italian sausages
Steps: set the oven at the right temp from directions on 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<s> [INST] 

# Recipe Assistant

## Context
You are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.

## Retrieved Recipes
The following recipes have been retrieved as most relevant to the user's query:

Document 0
Name: semi copycat sporky pizza bertucci
Description: this was my favorite pizza when i lived in boston.  ive tried to get our local pizza places to make and sell it, but to no avail. they just make the same old - same old pizza!
i have not included oven temp or time to cook because it all depends on the pizza you buy.
Ingredients: frozen cheese pizza, tomatoes, ricotta cheese, mozzarella cheese, olive oil, italian sausages
Steps: set the oven at the right temp from directions on pizza box, spri